# Percolation Threshold Calculation and Following Analysis

In [2]:
import numpy as np
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
from shapely import wkt
import os
import seaborn as sns
import matplotlib
# font setting
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']  


## Preparation: Find cbgs in the  water
This is because there are some cgbs that are in the water in our original dataset (id_dict and flow matrix). This phenomenon is mostly in NewYork.

In [ ]:
cbgs = gpd.read_file(r".\arcgis project\cbgs\cbgs_of_cities\NewYork\New_York_city.shp")
with open(r'.\data\Mobility\id_dict_1.pkl', 'rb') as f:
    id_dict = pickle.load(f)
    
print(cbgs.info())
print(len(id_dict.keys()))

# Extract the lines that not in cbgs but in id_dict
id_dict_keys = list(id_dict.keys())
id_dict_values = list(id_dict.values())
cbgs_keys = list(cbgs['CBG_Code'].values)
cbgs_in_water = []
count = 0
for i in id_dict_keys:
    if id_dict[id_dict_keys[i]] not in cbgs_keys:
        cbgs_in_water.append(i)
        count += 1
#cbgs_in_water

## Percolation Threshold Calculation

In [18]:
import numpy as np
import networkx as nx
import pandas as pd

def minimum_flows_for_strongly_connectivity(flow_matrix, percentile=2):
    """The algorithm to find the smallest k such that the subgraph remains strongly connected
       after removing the nodes with the smallest degree.
       
    Args:
        flow_matrix (numpy array): Flow matrix of the graph.

    Returns:
        ks: List of the smallest k such that the subgraph remains strongly connected.
    """
    ks = []
    # Function to check if the graph is strongly connected when only top k edges are kept for each node
    def check_strongly_connected_k(graph, k):
        # Create a new graph which only keeps top k outgoing edges for each node
        new_graph = nx.DiGraph()
        for node in graph.nodes():
            edges = sorted(graph.out_edges(node, data=True), key=lambda x: x[2]['weight'], reverse=True)[:k]
            new_graph.add_edges_from(edges)
        # Check if the new graph is still strongly connected
        return nx.is_strongly_connected(new_graph)
    
    # Set the diagonal elements to zero
    np.fill_diagonal(flow_matrix, 0)

    # Create directed graph from flow matrix
    G = nx.from_numpy_array(flow_matrix, create_using=nx.DiGraph)

    # Convert graph to numpy adjacency matrix
    adj_matrix = nx.adjacency_matrix(G)
    numpy_array = adj_matrix.toarray()
    # Convert the original flows to binary
    flows_binary = (numpy_array > 0).astype(int)
    # Calculate the degrees based on the new definition
    degrees_all = np.sum(np.logical_or(flows_binary, flows_binary.T), axis=1)

    # Determine the 2% degree threshold
    percentile_threshold = np.percentile(degrees_all, percentile)

    # find the nodes whose degree are in the lowest 2%
    nodes_to_remove = np.where(degrees_all <= percentile_threshold)[0]
    print('Number of nodes to remove: ', len(nodes_to_remove))

    # Remove the nodes from the matrix
    reduced_matrix = np.delete(numpy_array, nodes_to_remove, axis=0)
    reduced_matrix = np.delete(reduced_matrix, nodes_to_remove, axis=1)

    # Create directed graph from the reduced matrix
    reduced_G = nx.from_numpy_array(reduced_matrix, create_using=nx.DiGraph)
    
    # Ensure the graph is strongly connected
    scc = list(nx.strongly_connected_components(reduced_G))
    strongly_connected_G = reduced_G.subgraph(max(scc, key=len))
    num_units = len(strongly_connected_G.nodes())

    # Binary search variables
    low = 1
    high = num_units

    while low < high:
        mid = low + (high - low) // 2

        # Check if the graph is strongly connected for the current mid
        if check_strongly_connected_k(strongly_connected_G, mid):
            high = mid
        else:
            low = mid + 1

    # Final check if the graph is strongly connected for the low/high point
    if check_strongly_connected_k(strongly_connected_G, low):
        print(low)
        ks.append(low)
    else:
        print(-1)
        ks.append(-1)

    return ks


In [ ]:
# Calculate the threshold for each city
cities = ['New York, NY', 'Los Angeles, CA', 'Chicago, IL','Phoenix, AZ', 'Philadelphia, PA', 'San Diego, CA', 'Dallas, TX', 'San Jose, CA']
city_list = list(range(1,9))
delete_percent = 2
df_ks = pd.DataFrame(columns = ['year','month']+city_list)

for year in range(2018,2022):
    for month in range(1,13):
        df_ks.loc[len(df_ks),['year', 'month']] = [year, month]
        index = len(df_ks)-1
        for city in city_list:
            if os.path.exists(r'.\data\Mobility\cbg_visit_{year_}-{month_}_{city_}.npy'.format(year_ = year, month_ = str(month).zfill(2), city_ = city)):
                flow_matrix = np.load(r'.\data\Mobility\cbg_visit_{year_}-{month_}_{city_}.npy'.format(year_ = year, month_ = str(month).zfill(2), city_ = city))
                print(city, cities[city-1])
                # Remove the cbgs in the water in New York
                if city == 1:
                    flow_matrix = np.delete(flow_matrix, cbgs_in_water, axis=0)
                    flow_matrix = np.delete(flow_matrix, cbgs_in_water, axis=1)
                ks = minimum_flows_for_strongly_connectivity(flow_matrix, delete_percent)
                df_ks.loc[index, city] = ks[0]
                print(ks)

df_ks.columns =['year','month'] + cities
df_ks.to_csv('k_select_US_{}percent.csv'.format(delete_percent))

## Calculation of network metrics

In [ ]:
# Calculation of the clustering coefficient
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import networkx as nx
import geopy.distance
import time
from tqdm import tqdm

def get_k_subgraph(strongly_connected_G, k, node_to_cbg, cbg_to_centroid):
    new_graph = nx.DiGraph()
    edge_count = 0
    # 添加边并计算距离
    for node in strongly_connected_G.nodes():
        edges = sorted(strongly_connected_G.out_edges(node, data=True), key=lambda x: x[2]['weight'], reverse=True)[:k]
        for edge in edges:
            u, v, data = edge
            u_cbg = node_to_cbg.get(u)
            v_cbg = node_to_cbg.get(v)
            new_graph.add_edge(u, v, weight=data['weight'])
    return new_graph


cities = ['New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Phoenix, AZ', 'Philadelphia, PA', 'San Diego, CA', 'Dallas, TX', 'San Jose, CA']
results_df = pd.DataFrame(columns=['year', 'month', 'city', 'k', 'avg_distance','cluster_coeff'])

all_ks = [5, 10, 15, 20, 30, 50, 70, 90, 110, 130, 150, 200, 250, 300, 400, 500,  600, 900, 950, 1200, 1500, 2000, 2500]
# Load the files
percentile = 2
for city in range(1,9):
    # Load the files
    # delete results_df 
    del results_df
    results_df = pd.DataFrame(columns=['year', 'month', 'city', 'k','cluster_coeff'])
    with open(r"./data/Mobility/id_dict_{}.pkl".format(city), 'rb') as f:
        node_to_cbg = pickle.load(f)
    city_name = cities[city-1].split(',')[0].replace(' ', '_')
    gdf = gpd.read_file(r'./arcgis project/cbgs/cbgs_of_cities/{}_city.shp'.format(city_name))
    # save precomputed_distances
    for year in range(2018, 2022):
        for month in range(1,13):
            start = time.time()
            mobility_matrix = np.load(r'./data/Mobility/cbg_visit_{y}-{m}_{c}.npy'.format(y=year, m=str(month).zfill(2), c=city))
            # ks is the list that maximum element smaller than  gdf.shape[0] from all ks
            ks = [x for x in all_ks if x < gdf.shape[0]-100]

            # Remove the cbgs in the water in New York
            if city == 1:
                reduced_nodes = not_in_cbgs
                # Remove the nodes from the matrix
                flow_matrix = np.delete(mobility_matrix, reduced_nodes, axis=0)
                flow_matrix = np.delete(flow_matrix, reduced_nodes, axis=1)
            else:
                flow_matrix = mobility_matrix

            np.fill_diagonal(flow_matrix, 0)
            numpy_array = flow_matrix
            flows_binary = (numpy_array > 0).astype(int)
            degrees_all = np.sum(np.logical_or(flows_binary, flows_binary.T), axis=1)

            percentile_threshold = np.percentile(degrees_all, percentile)
            nodes_to_remove = np.where(degrees_all <= percentile_threshold)[0]

            original_indices = np.arange(numpy_array.shape[0])
            preserved_indices = np.delete(original_indices, nodes_to_remove)

            reduced_matrix = np.delete(numpy_array, nodes_to_remove, axis=0)
            reduced_matrix = np.delete(reduced_matrix, nodes_to_remove, axis=1)
            reduced_G = nx.from_numpy_array(reduced_matrix, create_using=nx.DiGraph)

            scc = list(nx.strongly_connected_components(reduced_G))
            max_scc = max(scc, key=len)
            preserved_indices = [index for index in preserved_indices if index in max_scc]
            strongly_connected_G = reduced_G.subgraph(max(scc, key=len))
            # 更新 node_to_cbg 映射
            updated_node_to_cbg = {new_index: node_to_cbg[original_index] for new_index, original_index in enumerate(preserved_indices)}

            # 初始化用于存储结果的数组
            avg_distances = []
            pre = time.time()
            print("pre time: ", pre-start)
            for k in tqdm(ks):    
                #mid1 = time.time()
                graph = get_k_subgraph(strongly_connected_G, k, updated_node_to_cbg, cbg_to_centroid)
                # 计算平均聚类系数
                avg_cluster_coeff = nx.average_clustering(graph.to_undirected())
                average_cluster_coeffs.append(avg_cluster_coeff)
            # 将结果存储到 DataFrame
            for i in range(len(ks)):
                results_df.loc[len(results_df)] = [year, month, city, ks[i], average_cluster_coeffs[i]]
    results_df.to_csv('k_select_avg_distance_cluster_{}.csv'.format(city))

## Calculation of the effective number of destinations (END)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

cities = ['New York, NY', 'Los Angeles, CA', 'Chicago, IL','Phoenix, AZ', 'Philadelphia, PA', 'San Diego, CA', 'Dallas, TX', 'San Jose, CA']
city_list = list(range(1,9))

df_effec_dest = pd.DataFrame(columns=["year","month"]+cities)

for year in range(2018,2022):
    for month in range(1,13):
        mean_lst = [year, month]
        for city in city_list:
            flow_matrix = np.load(r'.\data\Mobility\cbg_visit_{year_}-{month_}_{city_}.npy'.format(year_ = year, month_ = str(month).zfill(2), city_ = city))
            #print(city, cities[city-1])
            
            effective_dests = []
            for i in range(flow_matrix.shape[0]):

                sum = np.sum(flow_matrix[i])
                if sum == 0:
                    continue
                tmp_vec = flow_matrix[i]/sum
                effec_dest = 1 / np.sum(tmp_vec*tmp_vec)
                effective_dests.append(effec_dest)
            mean = np.mean(effective_dests)
            mean_lst.append(mean)
            print(cities[city-1], mean)
        df_effec_dest.loc[len(df_effec_dest)] = mean_lst
df_effec_dest.to_csv(r'.\results\effective_destinations.csv')